Comments for myself:

TOFINDOUT:
1. How to add to plots without retyping the whole thing
2. How to make    cep_gr.aggregate(lambda cp: smf.ols(formula = 'W ~ logP1', data = cp).fit())     work; Sotiria said I should make a function out of it.
3. How to extract the value of the t_test from the smf_ols_object.summary()
4. Try out:
C = np.logspace(-2, 2, 10)
scores = [cross_val_score(SVC(C=Ci), X, y, cv=5, scoring='precision').mean() for Ci in C]
definition apparently of an array in a nice elegant form

TODO:
1. add kernel smoother to the histograms
2. check the qq-plots for the other distributions (especially Poisson which is discrete) how to do it? Give the template 
3. give the command how to extract parameter significance table (t-test values) from the 

1. Distributions: 
how to get to know better the distribution of the data?

1.1. Probability distributions

In [2]:
import scipy
import pandas as pd
import numpy as np
import math
import statsmodels.api as sm
from matplotlib import pyplot as plt

Ex1. First, simulate 9 times 200 standard random normal variables, and inspect the variations in the qq-plots. 

In [ ]:
fig = plt.figure(1)
for ii in range(1, 10):
    rv = scipy.stats.norm.rvs(0, 1, size = 200)
    ax = fig.add_subplot(3,3,ii)
    sm.qqplot(rv, line = 's', ax = ax)
plt.show()

Repeat the exercise using random normal variables with mean 3 and standard deviation 0.4, using now 3 times each of 4,20 and 200.

The automatically added line on the qq-plot is estimated by taking the empirical mean and square-root empirical variance of the sample; these are the moment estimators of the parameters of a normal sample. For a normal distribution, if X ~ N(0,1), then the variable  Y = m + sX ~ N(m, s^2). Therefore, if you plotted your sample Y against standard normal quantiles, a line using the true mean m as intersect and the true standard deviation s as slope should represent the truth, and probably should fit the sample well. 

Add a line with intersection 3 and slope 0.4 (representing the n = Infty perfect sample). Are the empirical mean and variance good estimators of the population mean and variance?   

In [ ]:
x = np.arange(-3.,3.,0.01)
y = 3. + 0.4*x
nsample = [4,4,4,20,20,20,200,200,200]
fig = plt.figure(2)
for ii in range(0,9):
    rv = scipy.stats.norm.rvs(3., 0.4, size = nsample[ii])
    ax = fig.add_subplot(3,3,ii+1)
    sm.qqplot(rv, line = 's', ax = ax)
    line, = ax.plot(x, y, color='c')
plt.show()

The qq-plot provides a powerful check of distributional assumptions. Use the template below to see the following distributions: Cauchy (heavy-tailed), chi-squared (much used), beta(0.5,2) (restricted to the interval [0,1]; contains the uniform distribution as beta(1,1)) and two Poisson distributions, with mean 3 and 250. 

This time, we use the option fit = True in sm.qqplot, so that the sample is standardized by its mean and standard error before plotting. 

Replace the names of the random distribution in the codes below with those listed above (check the parameters at http://docs.scipy.org/doc/scipy/reference/stats.html). Compare the tail behavior on the plot of the density and on the qq-plot. 

In [17]:
x = np.linspace(scipy.stats.cauchy.ppf(0.005), scipy.stats.cauchy.ppf(0.995), 200)
#x = np.arange(scipy.stats.poisson.ppf(0.005, mu=250), scipy.stats.poisson.ppf(0.995, mu=250), 1)

rv = scipy.stats.poisson.rvs(size = 400, mu=250)

fig = plt.figure(3)
ax1 = fig.add_subplot(121)
plt.hist(rv, 20, normed=1, facecolor='y', alpha=0.75)
ax1.plot(x, scipy.stats.cauchy.pdf(x),'b-', lw=2)
#ax1.vlines(x, 0, scipy.stats.poisson.pmf(x, mu=250), colors='b', lw=5, alpha=0.2)
ax2 = fig.add_subplot(122)
sm.qqplot(rv, line = 's', ax = ax2)

plt.show()

In [14]:
print x
print scipy.stats.poisson.pmf(x, mu=250)

[ 214.]
[ 0.00178463]


1. Distributions: 

1.2. Estimating the parameters and an (incomplete) collection of frequent problems

We will walk through some of the simplest complications that a set of photometric measurements of a constant star can show up, and see how to recognize them by the means of the quantile-quantile plot. 


In [ ]:
dfr = pd.read_csv("/Users/mariasuveges/Documents/presentations/Chicago/IntroStat_demo.csv")

In [ ]:
dfr['mag.outlier'].quantile([0.,0.33,0.66,1.0])

In [ ]:
dfr['mag.outlier'].min()

In [ ]:
fig = plt.figure(1)
plt.subplot(121)
n, bins, patches = plt.hist(dfr['mag.outlier'], 12, normed=1, facecolor='y', alpha=0.75)
ax = fig.add_subplot(122)
sm.qqplot(dfr['mag.outlier'], line = 's', ax = ax)

plt.show()

Replace the line by one defined using the quantile estimators (a line passing through the first and third quartiles).

In [ ]:
fig = plt.figure(1)
plt.subplot(121)
n, bins, patches = plt.hist(dfr['mag.outlier'], 12, normed=1, facecolor='y', alpha=0.75)
ax = fig.add_subplot(122)
sm.qqplot(dfr['mag.outlier'], line = 'q', ax = ax)
plt.show()

Some robust (regression) models, M-estimators: http://statsmodels.sourceforge.net/stable/rlm.html

In [ ]:
def std_fn(x, mean, std):
    res = (x-mean) / std
    return res

w = dfr['mag5.error']**(-2)
m = np.average(dfr['mag5'], weights = w)
std5 = std_fn(x = dfr['mag5'], mean = m, std = dfr['mag5.error'])

fig = plt.figure(1)
plt.subplot(121)
n, bins, patches = plt.hist(std5, 12, normed=1, facecolor='grey', alpha=0.75)
ax = fig.add_subplot(122)
sm.qqplot(std5, line = '45', ax = ax)

plt.show()

In [ ]:
fig = plt.figure(1)

plt.plot(dfr['time'], dfr['mag5'], 'ro')

plt.xlabel('Time')
plt.ylabel('Magnitude')
# this is just to extend a bit the plotting area, so that no points fell exactly on the border:
mn, mx = sorted(dfr['time'])[::len(dfr['time'])-1]
plt.xlim(mn - 0.05*(mx-mn), mx + 0.05*(mx-mn))

plt.show()

In [20]:
cep = pd.read_csv("/Users/mariasuveges/Documents/presentations/Chicago/Cepheids.csv")

In [4]:
cep[1:10]

,name,RA0,Decl0,Mode,Cloud,W,logP1,VI
1,OGLE-LMC-CEP-0005,4.592089,-69.73497,F,LMC,13.49540,0.749122,0.752
2,OGLE-LMC-CEP-0012,4.630203,-67.21442,F,LMC,14.54210,0.424912,0.598
3,OGLE-LMC-CEP-0016,4.642367,-67.64631,F,LMC,12.03300,1.021456,1.080
4,OGLE-LMC-CEP-0017,4.649119,-69.68839,F,LMC,14.34215,0.565524,0.647
5,OGLE-LMC-CEP-0018,4.649717,-68.95689,F,LMC,13.93705,0.607225,0.829
6,OGLE-LMC-CEP-0021,4.658600,-69.26183,F,LMC,13.53005,0.737032,0.769
7,OGLE-LMC-CEP-0023,4.679856,-68.98564,F,LMC,15.21055,0.230915,0.719
8,OGLE-LMC-CEP-0025,4.681769,-69.11508,F,LMC,14.08130,0.572116,0.814
9,OGLE-LMC-CEP-0026,4.686733,-68.79975,F,LMC,14.51275,0.410047,0.615


In [21]:
i_lmc = cep['Cloud'] == "LMC"
i_fu = cep['Mode'] == "F"



In [ ]:
fig = plt.figure(4)
ax1 = fig.add_subplot(211)
plt.plot(cep[i_lmc & i_fu]['logP1'], cep[i_lmc & i_fu]['W'], 'b*', alpha=0.3, label = 'LMC')
plt.plot(cep[-i_lmc & i_fu]['logP1'], cep[-i_lmc & i_fu]['W'], 'r*', alpha=0.3, label = 'SMC')
plt.legend(loc = 'best', numpoints = 1)
ax2 = fig.add_subplot(212)
plt.plot(cep[i_lmc & -i_fu]['logP1'], cep[i_lmc & -i_fu]['W'], 'b*', alpha=0.3, label = 'LMC')
plt.plot(cep[-i_lmc & -i_fu]['logP1'], cep[-i_lmc & -i_fu]['W'], 'r*', alpha=0.3, label = 'SMC')
plt.legend(loc = 'best', numpoints = 1)

plt.show()

In [22]:
import statsmodels.formula.api as smf

In [ ]:
cep_gr = cep.groupby(['Cloud','Mode'])

In [ ]:
cep_gr.groups

In [ ]:
cep_gr.get_group(('LMC','F'))

In [ ]:
lmfit_lmc_fu = smf.ols(formula = 'W ~ logP1', data = cep_gr.get_group(('LMC','F'))).fit()

In [ ]:
print lmfit_lmc_fu.summary()

In [ ]:
# ez nem mukodik, nem értem, miért
cep_gr.aggregate(lambda cp: smf.ols(formula = 'W ~ logP1', data = cp).fit())

In [23]:
lmfit_lmc_fu = smf.ols(formula = 'W ~ logP1', data = cep, subset = i_lmc & i_fu).fit()
lmfit_lmc_fo = smf.ols(formula = 'W ~ logP1', data = cep, subset = i_lmc & -i_fu).fit()

lmfit_smc_fu = smf.ols(formula = 'W ~ logP1', data = cep, subset = -i_lmc & i_fu).fit()
lmfit_smc_fo = smf.ols(formula = 'W ~ logP1', data = cep, subset = -i_lmc & -i_fu).fit()

In [24]:
c_tmp = cep

In [25]:
c_tmp['resid0'] = np.zeros(c_tmp.shape[0])

In [26]:
c_tmp.loc[(i_lmc & i_fu),'resid0'] = lmfit_lmc_fu.resid
c_tmp.loc[(-i_lmc & i_fu),'resid0'] = lmfit_smc_fu.resid
c_tmp.loc[(i_lmc & -i_fu),'resid0'] = lmfit_lmc_fo.resid
c_tmp.loc[(-i_lmc & -i_fu),'resid0'] = lmfit_smc_fo.resid

In [16]:
c_tmp[1:10]

,name,RA0,Decl0,Mode,Cloud,W,logP1,VI,resid0
1,OGLE-LMC-CEP-0005,4.592089,-69.73497,F,LMC,13.49540,0.749122,0.752,0.094023
2,OGLE-LMC-CEP-0012,4.630203,-67.21442,F,LMC,14.54210,0.424912,0.598,0.062439
3,OGLE-LMC-CEP-0016,4.642367,-67.64631,F,LMC,12.03300,1.021456,1.080,-0.462625
4,OGLE-LMC-CEP-0017,4.649119,-69.68839,F,LMC,14.34215,0.565524,0.647,0.330146
5,OGLE-LMC-CEP-0018,4.649717,-68.95689,F,LMC,13.93705,0.607225,0.829,0.063738
6,OGLE-LMC-CEP-0021,4.658600,-69.26183,F,LMC,13.53005,0.737032,0.769,0.088461
7,OGLE-LMC-CEP-0023,4.679856,-68.98564,F,LMC,15.21055,0.230915,0.719,0.085676
8,OGLE-LMC-CEP-0025,4.681769,-69.11508,F,LMC,14.08130,0.572116,0.814,0.091221
9,OGLE-LMC-CEP-0026,4.686733,-68.79975,F,LMC,14.51275,0.410047,0.615,-0.016350


In [7]:
lmfit_lmc_fu.params

Intercept    15.892871
logP1        -3.325885
dtype: float64

In [27]:
# dfmi.loc[:,('one','second')] = value
c_tmp = pd.DataFrame(columns = ['resid0','fitted0'], dtype = float)
c_tmp2 = pd.DataFrame.add(cep, other = c_tmp, axis = "columns", level = 0)
c_tmp2[['name','RA0','Decl0','Mode','Cloud','logP1','VI','W']] = cep[['name','RA0','Decl0','Mode','Cloud','logP1','VI','W']]
c_tmp2.loc[(i_lmc & i_fu),'resid0'] = lmfit_lmc_fu.resid
c_tmp2.loc[(-i_lmc & i_fu),'resid0'] = lmfit_smc_fu.resid
c_tmp2.loc[(i_lmc & -i_fu),'resid0'] = lmfit_lmc_fo.resid
c_tmp2.loc[(-i_lmc & -i_fu),'resid0'] = lmfit_smc_fo.resid
c_tmp2.loc[(i_lmc & i_fu),'fitted0'] = lmfit_lmc_fu.fittedvalues
c_tmp2.loc[(-i_lmc & i_fu),'fitted0'] = lmfit_smc_fu.fittedvalues
c_tmp2.loc[(i_lmc & -i_fu),'fitted0'] = lmfit_lmc_fo.fittedvalues
c_tmp2.loc[(-i_lmc & -i_fu),'fitted0'] = lmfit_smc_fo.fittedvalues

In [28]:
c_tmp2.iloc[0:10]

,Cloud,Decl0,Mode,RA0,VI,W,fitted0,logP1,name,resid0
0,LMC,-69.81933,F,4.529733,0.740,14.52500,14.250240,0.493893,OGLE-LMC-CEP-0002,0.274760
1,LMC,-69.73497,F,4.592089,0.752,13.49540,13.401377,0.749122,OGLE-LMC-CEP-0005,0.094023
2,LMC,-67.21442,F,4.630203,0.598,14.54210,14.479661,0.424912,OGLE-LMC-CEP-0012,0.062439
3,LMC,-67.64631,F,4.642367,1.080,12.03300,12.495625,1.021456,OGLE-LMC-CEP-0016,-0.462625
4,LMC,-69.68839,F,4.649119,0.647,14.34215,14.012004,0.565524,OGLE-LMC-CEP-0017,0.330146
5,LMC,-68.95689,F,4.649717,0.829,13.93705,13.873312,0.607225,OGLE-LMC-CEP-0018,0.063738
6,LMC,-69.26183,F,4.658600,0.769,13.53005,13.441589,0.737032,OGLE-LMC-CEP-0021,0.088461
7,LMC,-68.98564,F,4.679856,0.719,15.21055,15.124874,0.230915,OGLE-LMC-CEP-0023,0.085676
8,LMC,-69.11508,F,4.681769,0.814,14.08130,13.990079,0.572116,OGLE-LMC-CEP-0025,0.091221
9,LMC,-68.79975,F,4.686733,0.615,14.51275,14.529100,0.410047,OGLE-LMC-CEP-0026,-0.016350


In [ ]:
logp_tmp = np.linspace(cep['logP1'].min(), cep['logP1'].max(), 500)

fig = plt.figure(4)
ax1 = fig.add_subplot(211)
plt.plot(cep[i_lmc & i_fu]['logP1'], cep[i_lmc & i_fu]['W'], 'c*', alpha=0.2, label = 'LMC')
plt.plot(cep[-i_lmc & i_fu]['logP1'], cep[-i_lmc & i_fu]['W'], 'r*', alpha=0.2, label = 'SMC')
plt.plot(logp_tmp, lmfit_lmc_fu.params['Intercept'] + logp_tmp * lmfit_lmc_fu.params['logP1'], 'blue', lw = 1)
plt.plot(logp_tmp, lmfit_smc_fu.params['Intercept'] + logp_tmp * lmfit_smc_fu.params['logP1'], 'brown', lw = 1)
plt.legend(loc = 'best', numpoints = 1)
ax2 = fig.add_subplot(212)
plt.plot(cep[i_lmc & -i_fu]['logP1'], cep[i_lmc & -i_fu]['W'], 'c*', alpha=0.2, label = 'LMC')
plt.plot(cep[-i_lmc & -i_fu]['logP1'], cep[-i_lmc & -i_fu]['W'], 'r*', alpha=0.2, label = 'SMC')
plt.legend(loc = 'best', numpoints = 1)
plt.plot(logp_tmp, lmfit_lmc_fo.params['Intercept'] + logp_tmp * lmfit_lmc_fo.params['logP1'], 'blue', lw = 1)
plt.plot(logp_tmp, lmfit_smc_fo.params['Intercept'] + logp_tmp * lmfit_smc_fo.params['logP1'], 'brown', lw = 1)

plt.show()

In [ ]:
mn, mx = sorted(c_tmp2['resid0'])[::len(c_tmp2['resid0'])-1]

fig = plt.figure(1)

ax1 = fig.add_subplot(221)
sm.qqplot(c_tmp2[i_lmc & i_fu]['resid0'], line = 's', ax = ax1)
plt.title("LMC FU")
plt.ylim(mn - 0.05*(mx-mn), mx + 0.05*(mx-mn))
ax2 = fig.add_subplot(222)
sm.qqplot(c_tmp2[-i_lmc & i_fu]['resid0'], line = 's', ax = ax2)
plt.title("SMC FU")
plt.ylim(mn - 0.05*(mx-mn), mx + 0.05*(mx-mn))
ax3 = fig.add_subplot(223)
sm.qqplot(c_tmp2[i_lmc & -i_fu]['resid0'], line = 's', ax = ax3)
plt.title("LMC FO")
plt.ylim(mn - 0.05*(mx-mn), mx + 0.05*(mx-mn))
ax4 = fig.add_subplot(224)
sm.qqplot(c_tmp2[-i_lmc & -i_fu]['resid0'], line = 's', ax = ax4)
plt.title("SMC FO")
plt.ylim(mn - 0.05*(mx-mn), mx + 0.05*(mx-mn))

plt.show()

Map, coloured according to the sign of the residuals:

In [ ]:
i_posresid = (c_tmp2['resid0'] > 0)
    
fig = plt.figure(1)

fig.add_subplot(221)
plt.plot(c_tmp2[-i_lmc & i_fu & i_posresid]['RA0'], c_tmp2[-i_lmc & i_fu & i_posresid]['Decl0'], 'bo', alpha=0.2)
plt.plot(c_tmp2[-i_lmc & i_fu & -i_posresid]['RA0'], c_tmp2[-i_lmc & i_fu & -i_posresid]['Decl0'], 'yo', alpha=0.2)
plt.title('SMC FU')

fig.add_subplot(222)
plt.plot(c_tmp2[i_lmc & i_fu & i_posresid]['RA0'], c_tmp2[i_lmc & i_fu & i_posresid]['Decl0'], 'bo', alpha=0.2)
plt.plot(c_tmp2[i_lmc & i_fu & -i_posresid]['RA0'], c_tmp2[i_lmc & i_fu & -i_posresid]['Decl0'], 'yo', alpha=0.2)
plt.title('LMC FU')

fig.add_subplot(223)
plt.plot(c_tmp2[-i_lmc & -i_fu & i_posresid]['RA0'], c_tmp2[-i_lmc & -i_fu & i_posresid]['Decl0'], 'bo', alpha=0.2)
plt.plot(c_tmp2[-i_lmc & -i_fu & -i_posresid]['RA0'], c_tmp2[-i_lmc & -i_fu & -i_posresid]['Decl0'], 'yo', alpha=0.2)
plt.title('SMC FU')

fig.add_subplot(224)
plt.plot(c_tmp2[i_lmc & -i_fu & i_posresid]['RA0'], c_tmp2[i_lmc & -i_fu & i_posresid]['Decl0'], 'bo', alpha=0.2)
plt.plot(c_tmp2[i_lmc & -i_fu & -i_posresid]['RA0'], c_tmp2[i_lmc & -i_fu & -i_posresid]['Decl0'], 'yo', alpha=0.2)
plt.title('LMC FU')

plt.show()

Residuals against fitted value

In [ ]:
fig = plt.figure(1)

fig.add_subplot(221)
plt.plot(c_tmp2[-i_lmc & i_fu]['fitted0'], c_tmp2[-i_lmc & i_fu]['resid0'], 'g*', alpha=0.2)
plt.hlines(0, xmin = c_tmp2[-i_lmc & i_fu]['fitted0'].min(), xmax = c_tmp2[-i_lmc & i_fu]['fitted0'].max(), lw = 2)
plt.title('SMC FU')

fig.add_subplot(222)
plt.plot(c_tmp2[i_lmc & i_fu]['fitted0'], c_tmp2[i_lmc & i_fu]['resid0'], 'g*', alpha=0.2)
plt.hlines(0, xmin = c_tmp2[-i_lmc & i_fu]['fitted0'].min(), xmax = c_tmp2[-i_lmc & i_fu]['fitted0'].max(), lw = 2)
plt.title('LMC FU')

fig.add_subplot(223)
plt.plot(c_tmp2[-i_lmc & -i_fu]['fitted0'], c_tmp2[-i_lmc & -i_fu]['resid0'], 'g*', alpha=0.2)
plt.hlines(0, xmin = c_tmp2[-i_lmc & -i_fu]['fitted0'].min(), xmax = c_tmp2[-i_lmc & -i_fu]['fitted0'].max(), lw = 2)
plt.title('SMC FO')

fig.add_subplot(224)
plt.plot(c_tmp2[i_lmc & -i_fu]['fitted0'], c_tmp2[i_lmc & -i_fu]['resid0'], 'g*', alpha=0.2)
plt.hlines(0, xmin = c_tmp2[i_lmc & -i_fu]['fitted0'].min(), xmax = c_tmp2[i_lmc & -i_fu]['fitted0'].max(), lw = 2)
plt.title('LMC FO')

plt.show()


Residuals against covariate

In [ ]:
fig = plt.figure(1)

fig.add_subplot(221)
plt.plot(c_tmp2[-i_lmc & i_fu]['logP1'], c_tmp2[-i_lmc & i_fu]['resid0'], 'g*', alpha=0.2)
plt.hlines(0, xmin = c_tmp2[-i_lmc & i_fu]['logP1'].min(), xmax = c_tmp2[-i_lmc & i_fu]['logP1'].max(), lw = 2)
plt.title('SMC FU')

fig.add_subplot(222)
plt.plot(c_tmp2[i_lmc & i_fu]['logP1'], c_tmp2[i_lmc & i_fu]['resid0'], 'g*', alpha=0.2)
plt.hlines(0, xmin = c_tmp2[-i_lmc & i_fu]['logP1'].min(), xmax = c_tmp2[-i_lmc & i_fu]['logP1'].max(), lw = 2)
plt.title('LMC FU')

fig.add_subplot(223)
plt.plot(c_tmp2[-i_lmc & -i_fu]['logP1'], c_tmp2[-i_lmc & -i_fu]['resid0'], 'g*', alpha=0.2)
plt.hlines(0, xmin = c_tmp2[-i_lmc & -i_fu]['logP1'].min(), xmax = c_tmp2[-i_lmc & -i_fu]['logP1'].max(), lw = 2)
plt.title('SMC FO')

fig.add_subplot(224)
plt.plot(c_tmp2[i_lmc & -i_fu]['logP1'], c_tmp2[i_lmc & -i_fu]['resid0'], 'g*', alpha=0.2)
plt.hlines(0, xmin = c_tmp2[i_lmc & -i_fu]['logP1'].min(), xmax = c_tmp2[i_lmc & -i_fu]['logP1'].max(), lw = 2)
plt.title('LMC FO')

plt.show()



Residuals vs VI

In [ ]:
fig = plt.figure(1)

fig.add_subplot(221)
plt.plot(c_tmp2[-i_lmc & i_fu]['VI'], c_tmp2[-i_lmc & i_fu]['resid0'], 'g*', alpha=0.2)
plt.hlines(0, xmin = c_tmp2[-i_lmc & i_fu]['VI'].min(), xmax = c_tmp2[-i_lmc & i_fu]['VI'].max(), lw = 2)
plt.title('SMC FU')

fig.add_subplot(222)
plt.plot(c_tmp2[i_lmc & i_fu]['VI'], c_tmp2[i_lmc & i_fu]['resid0'], 'g*', alpha=0.2)
plt.hlines(0, xmin = c_tmp2[-i_lmc & i_fu]['VI'].min(), xmax = c_tmp2[-i_lmc & i_fu]['VI'].max(), lw = 2)
plt.title('LMC FU')

fig.add_subplot(223)
plt.plot(c_tmp2[-i_lmc & -i_fu]['VI'], c_tmp2[-i_lmc & -i_fu]['resid0'], 'g*', alpha=0.2)
plt.hlines(0, xmin = c_tmp2[-i_lmc & -i_fu]['VI'].min(), xmax = c_tmp2[-i_lmc & -i_fu]['VI'].max(), lw = 2)
plt.title('SMC FO')

fig.add_subplot(224)
plt.plot(c_tmp2[i_lmc & -i_fu]['VI'], c_tmp2[i_lmc & -i_fu]['resid0'], 'g*', alpha=0.2)
plt.hlines(0, xmin = c_tmp2[i_lmc & -i_fu]['VI'].min(), xmax = c_tmp2[i_lmc & -i_fu]['VI'].max(), lw = 2)
plt.title('LMC FO')

plt.show()

Careful: VI versus logP1

In [ ]:
fig = plt.figure(1)

fig.add_subplot(221)
plt.plot(c_tmp2[-i_lmc & i_fu]['logP1'], c_tmp2[-i_lmc & i_fu]['VI'], 'g*', alpha=0.2)
plt.title('SMC FU')

fig.add_subplot(222)
plt.plot(c_tmp2[i_lmc & i_fu]['logP1'], c_tmp2[i_lmc & i_fu]['VI'], 'g*', alpha=0.2)
plt.title('LMC FU')

fig.add_subplot(223)
plt.plot(c_tmp2[-i_lmc & -i_fu]['logP1'], c_tmp2[-i_lmc & -i_fu]['VI'], 'g*', alpha=0.2)
plt.title('SMC FO')

fig.add_subplot(224)
plt.plot(c_tmp2[i_lmc & -i_fu]['logP1'], c_tmp2[i_lmc & -i_fu]['VI'], 'g*', alpha=0.2)
plt.title('LMC FO')

plt.show()

In [29]:
vi_lmfit_lmc_fu = smf.ols(formula = 'VI ~ logP1', data = cep, subset = i_lmc & i_fu).fit()
vi_lmfit_lmc_fo = smf.ols(formula = 'VI ~ logP1', data = cep, subset = i_lmc & -i_fu).fit()

vi_lmfit_smc_fu = smf.ols(formula = 'VI ~ logP1', data = cep, subset = -i_lmc & i_fu).fit()
vi_lmfit_smc_fo = smf.ols(formula = 'VI ~ logP1', data = cep, subset = -i_lmc & -i_fu).fit()

In [31]:
vi_lmfit_smc_fo.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     VI   R-squared:                       0.122
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:                     226.5
Date:                Mon, 01 Aug 2016   Prob (F-statistic):           4.91e-48
Time:                        18:00:49   Log-Likelihood:                 1500.5
No. Observations:                1632   AIC:                            -2997.
Df Residuals:                    1630   BIC:                            -2986.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.5593      0.003    199.634      0.000         0.554     0.565
logP1          0.1712      0.011     15.051      0.000         0.149     0.194
==============================================================================
Omnibus:                     1312.692   Durbin-Watson:                   1.892
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            67973.071
Skew:                           3.338   Prob(JB):                         0.00
Kurtosis:                      33.904   Cond. No.                         4.84
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
c_tmp = pd.DataFrame(columns = ['resid_vi'], dtype = float)
c_tmp3 = pd.DataFrame.add(c_tmp2, other = c_tmp, axis = "columns", level = 0)
c_tmp3[['name','RA0','Decl0','Mode','Cloud','logP1','VI','W','fitted0','resid0']] = c_tmp2[['name','RA0','Decl0','Mode','Cloud','logP1','VI','W','fitted0','resid0']]
c_tmp3.loc[(i_lmc & i_fu),'resid_vi'] = vi_lmfit_lmc_fu.resid
c_tmp3.loc[(-i_lmc & i_fu),'resid_vi'] = vi_lmfit_smc_fu.resid
c_tmp3.loc[(i_lmc & -i_fu),'resid_vi'] = vi_lmfit_lmc_fo.resid
c_tmp3.loc[(-i_lmc & -i_fu),'resid_vi'] = vi_lmfit_smc_fo.resid

In [33]:
c_tmp3.iloc[0:10]

,Cloud,Decl0,Mode,RA0,VI,W,fitted0,logP1,name,resid0,resid_vi
0,LMC,-69.81933,F,4.529733,0.740,14.52500,14.250240,0.493893,OGLE-LMC-CEP-0002,0.274760,-0.026589
1,LMC,-69.73497,F,4.592089,0.752,13.49540,13.401377,0.749122,OGLE-LMC-CEP-0005,0.094023,-0.073301
2,LMC,-67.21442,F,4.630203,0.598,14.54210,14.479661,0.424912,OGLE-LMC-CEP-0012,0.062439,-0.152721
3,LMC,-67.64631,F,4.642367,1.080,12.03300,12.495625,1.021456,OGLE-LMC-CEP-0016,-0.462625,0.192053
4,LMC,-69.68839,F,4.649119,0.647,14.34215,14.012004,0.565524,OGLE-LMC-CEP-0017,0.330146,-0.136067
5,LMC,-68.95689,F,4.649717,0.829,13.93705,13.873312,0.607225,OGLE-LMC-CEP-0018,0.063738,0.036341
6,LMC,-69.26183,F,4.658600,0.769,13.53005,13.441589,0.737032,OGLE-LMC-CEP-0021,0.088461,-0.053519
7,LMC,-68.98564,F,4.679856,0.719,15.21055,15.124874,0.230915,OGLE-LMC-CEP-0023,0.085676,0.012905
8,LMC,-69.11508,F,4.681769,0.814,14.08130,13.990079,0.572116,OGLE-LMC-CEP-0025,0.091221,0.029417
9,LMC,-68.79975,F,4.686733,0.615,14.51275,14.529100,0.410047,OGLE-LMC-CEP-0026,-0.016350,-0.132302


In [34]:
lmfit_lmc_fu2 = smf.ols(formula = 'W ~ logP1 + resid_vi', data = c_tmp3, subset = i_lmc & i_fu).fit()
lmfit_lmc_fo2 = smf.ols(formula = 'W ~ logP1 + resid_vi', data = c_tmp3, subset = i_lmc & -i_fu).fit()

lmfit_smc_fu2 = smf.ols(formula = 'W ~ logP1 + resid_vi', data = c_tmp3, subset = -i_lmc & i_fu).fit()
lmfit_smc_fo2 = smf.ols(formula = 'W ~ logP1 + resid_vi', data = c_tmp3, subset = -i_lmc & -i_fu).fit()

In [35]:
print lmfit_lmc_fu2.summary()

                            OLS Regression Results                            
Dep. Variable:                      W   R-squared:                       0.962
Model:                            OLS   Adj. R-squared:                  0.962
Method:                 Least Squares   F-statistic:                 2.280e+04
Date:                Mon, 01 Aug 2016   Prob (F-statistic):               0.00
Time:                        18:01:09   Log-Likelihood:                 960.12
No. Observations:                1806   AIC:                            -1914.
Df Residuals:                    1803   BIC:                            -1898.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     15.8929      0.010   1584.215      0.0

In [36]:
print lmfit_lmc_fu.summary()

                            OLS Regression Results                            
Dep. Variable:                      W   R-squared:                       0.961
Model:                            OLS   Adj. R-squared:                  0.961
Method:                 Least Squares   F-statistic:                 4.456e+04
Date:                Mon, 01 Aug 2016   Prob (F-statistic):               0.00
Time:                        18:01:15   Log-Likelihood:                 939.63
No. Observations:                1806   AIC:                            -1875.
Df Residuals:                    1804   BIC:                            -1864.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     15.8929      0.010   1566.779      0.0

In [22]:
lmfit_lmc_fu1 = smf.ols(formula = 'W ~ logP1 + VI', data = c_tmp3, subset = i_lmc & i_fu).fit()
lmfit_lmc_fo1 = smf.ols(formula = 'W ~ logP1 + VI', data = c_tmp3, subset = i_lmc & -i_fu).fit()

lmfit_smc_fu1 = smf.ols(formula = 'W ~ logP1 + VI', data = c_tmp3, subset = -i_lmc & i_fu).fit()
lmfit_smc_fo1 = smf.ols(formula = 'W ~ logP1 + VI', data = c_tmp3, subset = -i_lmc & -i_fu).fit()

In [39]:
print lmfit_lmc_fu1.summary()

                            OLS Regression Results                            
Dep. Variable:                      W   R-squared:                       0.962
Model:                            OLS   Adj. R-squared:                  0.962
Method:                 Least Squares   F-statistic:                 2.280e+04
Date:                Tue, 26 Jul 2016   Prob (F-statistic):               0.00
Time:                        11:08:46   Log-Likelihood:                 960.12
No. Observations:                1806   AIC:                            -1914.
Df Residuals:                    1803   BIC:                            -1898.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     16.0083      0.021    778.718      0.0

In [32]:
c_tmp = pd.DataFrame(columns = ['resid2'], dtype = float)
c_tmp4 = pd.DataFrame.add(c_tmp3, other = c_tmp, axis = "columns")
c_tmp4[['name','RA0','Decl0','Mode','Cloud','logP1','VI','W','fitted0','resid0','resid_vi']] = c_tmp3[['name','RA0','Decl0','Mode','Cloud','logP1','VI','W','fitted0','resid0','resid_vi']]
c_tmp4.loc[(i_lmc & i_fu),'resid2'] = lmfit_lmc_fu2.resid
c_tmp4.loc[(-i_lmc & i_fu),'resid2'] = lmfit_smc_fu2.resid
c_tmp4.loc[(i_lmc & -i_fu),'resid2'] = lmfit_lmc_fo2.resid
c_tmp4.loc[(-i_lmc & -i_fu),'resid2'] = lmfit_smc_fo2.resid

In [33]:
c_tmp4.iloc[0:10]

,Cloud,Decl0,Mode,RA0,VI,W,fitted0,logP1,name,resid0,resid2,resid_vi
0,LMC,-69.81933,F,4.529733,0.740,14.52500,14.250240,0.493893,OGLE-LMC-CEP-0002,0.274760,0.270060,-0.026589
1,LMC,-69.73497,F,4.592089,0.752,13.49540,13.401377,0.749122,OGLE-LMC-CEP-0005,0.094023,0.081067,-0.073301
2,LMC,-67.21442,F,4.630203,0.598,14.54210,14.479661,0.424912,OGLE-LMC-CEP-0012,0.062439,0.035445,-0.152721
3,LMC,-67.64631,F,4.642367,1.080,12.03300,12.495625,1.021456,OGLE-LMC-CEP-0016,-0.462625,-0.428679,0.192053
4,LMC,-69.68839,F,4.649119,0.647,14.34215,14.012004,0.565524,OGLE-LMC-CEP-0017,0.330146,0.306096,-0.136067
5,LMC,-68.95689,F,4.649717,0.829,13.93705,13.873312,0.607225,OGLE-LMC-CEP-0018,0.063738,0.070162,0.036341
6,LMC,-69.26183,F,4.658600,0.769,13.53005,13.441589,0.737032,OGLE-LMC-CEP-0021,0.088461,0.079001,-0.053519
7,LMC,-68.98564,F,4.679856,0.719,15.21055,15.124874,0.230915,OGLE-LMC-CEP-0023,0.085676,0.087957,0.012905
8,LMC,-69.11508,F,4.681769,0.814,14.08130,13.990079,0.572116,OGLE-LMC-CEP-0025,0.091221,0.096421,0.029417
9,LMC,-68.79975,F,4.686733,0.615,14.51275,14.529100,0.410047,OGLE-LMC-CEP-0026,-0.016350,-0.039735,-0.132302


In [36]:
fig = plt.figure(1)

fig.add_subplot(221)
plt.plot(c_tmp2[-i_lmc & i_fu]['logP1'], c_tmp2[-i_lmc & i_fu]['resid0'], 'ko', alpha=0.6)
plt.plot(c_tmp4[-i_lmc & i_fu]['logP1'], c_tmp4[-i_lmc & i_fu]['resid2'], "mo", alpha=0.4)
#plt.hlines(0, xmin = c_tmp2[-i_lmc & i_fu]['logP1'].min(), xmax = c_tmp2[-i_lmc & i_fu]['logP1'].max(), lw = 2)
plt.title('SMC FU')

fig.add_subplot(222)
plt.plot(c_tmp2[i_lmc & i_fu]['logP1'], c_tmp2[i_lmc & i_fu]['resid0'], 'ko', alpha=0.6)
plt.plot(c_tmp4[i_lmc & i_fu]['logP1'], c_tmp4[i_lmc & i_fu]['resid2'], 'mo', alpha=0.4)
#plt.hlines(0, xmin = c_tmp2[-i_lmc & i_fu]['logP1'].min(), xmax = c_tmp2[-i_lmc & i_fu]['logP1'].max(), lw = 2)
plt.title('LMC FU')

fig.add_subplot(223)
plt.plot(c_tmp2[-i_lmc & -i_fu]['logP1'], c_tmp2[-i_lmc & -i_fu]['resid0'], 'ko', alpha=0.6)
plt.plot(c_tmp4[-i_lmc & -i_fu]['logP1'], c_tmp4[-i_lmc & -i_fu]['resid2'], 'mo', alpha=0.4)
#plt.hlines(0, xmin = c_tmp2[-i_lmc & -i_fu]['logP1'].min(), xmax = c_tmp2[-i_lmc & -i_fu]['logP1'].max(), lw = 2)
plt.title('SMC FO')

fig.add_subplot(224)
plt.plot(c_tmp2[i_lmc & -i_fu]['logP1'], c_tmp2[i_lmc & -i_fu]['resid0'], 'ko', alpha=0.6)
plt.plot(c_tmp4[i_lmc & -i_fu]['logP1'], c_tmp4[i_lmc & -i_fu]['resid2'], 'mo', alpha=0.4)
#plt.hlines(0, xmin = c_tmp2[i_lmc & -i_fu]['logP1'].min(), xmax = c_tmp2[i_lmc & -i_fu]['logP1'].max(), lw = 2)
plt.title('LMC FO')

plt.show()




In [29]:
lmfit_lmc_fo1.bic

-1154.2452576013848

In [30]:
lmfit_lmc_fo2.bic

-1154.2452576013848

In [37]:
    print lmfit_lmc_fo.bic
    print lmfit_lmc_fo2.bic
    print lmfit_lmc_fo1.bic

-1126.47644923
-1154.2452576
-1154.2452576


In [40]:
print lmfit_lmc_fu1.summary()
print lmfit_lmc_fu2.summary()

                            OLS Regression Results                            
Dep. Variable:                      W   R-squared:                       0.962
Model:                            OLS   Adj. R-squared:                  0.962
Method:                 Least Squares   F-statistic:                 2.280e+04
Date:                Tue, 26 Jul 2016   Prob (F-statistic):               0.00
Time:                        11:11:22   Log-Likelihood:                 960.12
No. Observations:                1806   AIC:                            -1914.
Df Residuals:                    1803   BIC:                            -1898.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     16.0083      0.021    778.718      0.0

In [36]:
?

Object `smf.summary` not found.


In [34]:
dir(lmfit_lmc_fu)

['HC0_se',
 'HC1_se',
 'HC2_se',
 'HC3_se',
 '_HCCM',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cache',
 '_data_attr',
 '_get_robustcov_results',
 '_is_nested',
 '_wexog_singular_values',
 'aic',
 'bic',
 'bse',
 'centered_tss',
 'compare_f_test',
 'compare_lm_test',
 'compare_lr_test',
 'condition_number',
 'conf_int',
 'conf_int_el',
 'cov_HC0',
 'cov_HC1',
 'cov_HC2',
 'cov_HC3',
 'cov_kwds',
 'cov_params',
 'cov_type',
 'df_model',
 'df_resid',
 'eigenvals',
 'el_test',
 'ess',
 'f_pvalue',
 'f_test',
 'fittedvalues',
 'fvalue',
 'get_influence',
 'get_robustcov_results',
 'initialize',
 'k_constant',
 'llf',
 'load',
 'model',
 'mse_model',
 'mse_resid',
 'mse_total',
 'nobs',
 'normalized_cov_params',
 'outlier_test',
 'params',
 'predict',
 'pvalues',


In [ ]:
help(pd.stats.math)